## Day 49 Lecture 1 Assignment

In this assignment, we will apply GMM (Gaussian Mixture Modeling) clustering to a dataset containing player-season statistics for NBA players from the past four years.

In [46]:
%matplotlib inline

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.mixture import GaussianMixture
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from scipy.special import entr

This dataset contains player-season statistics for NBA players from the past four years. Each row in this dataset represents a player's per-game averages for a single season. 

This dataset contains the following variables:

- Seas: season ('2019' = 2018-2019 season, '2018' = 2017-2018 season, etc.)
- Player: player name
- Pos: position
- Age: age
- Tm: team
- G: games played
- GS: games started
- MP: minutes played
- FG: field goals
- FGA: field goals attempted
- FG%: field goal percentage
- 3P: 3 pointers
- 3PA: 3 pointers attempted
- 3P%: 3 point percentage
- 2P: 2 pointers
- 2PA: 2 pointers attempted
- 2P%: 2 point percentage
- eFG%: effective field goal percentage
- FT: free throws
- FTA: free throws attempted
- FT%: free throw percentage
- ORB: offensive rebound
- DRB: defensive rebound
- TRB: total rebounds
- AST: assists
- STL: steals
- BLK: blocks
- TOV: turnovers
- PF: personal fouls
- PTS: points

Load the dataset.

In [11]:
# answer goes here

players = pd.read_csv('https://tf-assets-prod.s3.amazonaws.com/tf-curric/data-science/Data%20Sets%20Clustering/nba_player_seasons.csv')


The goal is to cluster these player-seasons to identify potential player "archetypes".  
The pre-processing steps will be identical to what we previously did for K-means.

Begin by removing players whose season did not meet one of the following criteria:
1. Started at least 20 games
2. Averaged at least 10 minutes per game

In [21]:
players_df = players.loc[players['GS'] >= 20]
players_df = players.loc[players['MP'] >= 10]

Choose a subset of numeric columns that is interesting to you from an "archetypal" standpoint. 

We will choose the following basic statistics: **points, total rebounds, assists, steals, blocks**, and **turnovers**, but you should feel free to choose other reasonable feature sets if you like. Be careful not to include too many dimensions (curse of dimensionality).

In [22]:
keep_cols = ['PTS', 'TRB', 'AST', 'STL', 'BLK', 'TOV']

X = players_df[keep_cols]

X

,PTS,TRB,AST,STL,BLK,TOV
0,5.3,1.5,0.6,0.5,0.2,0.5
1,1.7,2.5,0.8,0.1,0.4,0.4
2,3.2,1.8,1.9,0.4,0.1,0.8
3,13.9,9.5,1.6,1.5,1.0,1.7
4,8.9,7.3,2.2,0.9,0.8,1.5
...,...,...,...,...,...,...
2134,8.4,2.6,2.5,0.4,0.0,3.6
2137,7.3,1.8,0.6,0.4,0.1,0.6
2138,15.1,9.0,1.8,1.5,0.5,1.9
2139,8.7,6.2,1.0,0.8,0.9,0.9


Standardize the features in your dataset using scikit-learn's StandardScaler, which will set the mean of each feature to 0 and the variance to 1.

In [23]:
# answer goes here



from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

X_std = scaler.fit_transform(X)

Run both K-Means and Gaussian mixtures modeling twice, once with 3 cluster and once with 7 clusters. Print out the resulting means for all 4 scenarios (KM+3, GMM+3, KM+7, GMM+7). When printing the means, transform the scaled versions back into their corresponding unscaled values. 

What "archetypes" do you see? Are the archetypes identified by GMM similar to those identified by K-Means? How do the means of GMM differ from those of K-Means?

In [31]:
# answer goes here
result_means = {}

for i in [3,7]:
    k_means = KMeans(n_clusters=i)
    k_means.fit(X_std)
    result_means['KM+{}'.format(i)] = pd.DataFrame(scaler.inverse_transform(k_means.cluster_centers_), columns=X.columns)

    gmm = GaussianMixture(n_components=i)
    gmm.fit(X_std)
    result_means['GMM+{}'.format(i)] = pd.DataFrame(scaler.inverse_transform(gmm.means_), columns=X.columns)

In [32]:
result_means

{'KM+3':          PTS       TRB       AST       STL       BLK       TOV
 0   6.772920  2.849381  1.498584  0.553805  0.283894  0.877522
 1  17.829779  4.693382  5.204044  1.232353  0.417279  2.519853
 2  12.535457  7.197507  1.783102  0.822715  0.981717  1.488920,
 'GMM+3':          PTS       TRB       AST       STL       BLK       TOV
 0  13.159741  3.542396  3.920065  0.991215  0.288711  1.872719
 1   6.309249  2.900933  1.177581  0.486149  0.289141  0.772971
 2  11.867051  6.746418  1.801008  0.815653  0.939441  1.441006,
 'KM+7':          PTS       TRB       AST       STL       BLK       TOV
 0  23.677778  8.369444  7.588889  1.605556  0.877778  3.808333
 1   7.110438  4.738384  1.045455  0.523232  0.726599  0.888215
 2  16.413158  9.764474  2.002632  0.813158  1.809211  1.957895
 3  10.183820  2.891777  2.870027  0.842971  0.227851  1.403714
 4   5.400666  2.313977  1.076872  0.427621  0.195840  0.684359
 5  18.174342  4.415132  5.471711  1.284868  0.367105  2.538158
 6  13.255357

##

Predict the likelihood of each season belonging to one of the 3 clusters using the GMM model. Then, calculate the entropy for each set of predicted probabilities. 

We will use entropy as a measure of how confident we are in the predicted class label. If we had no confidence in our prediction, we would assign 33% probability to each class, while if we were totally confident, we would assign 100% to one class. Entropy would be at a maximum in the "no confidence" scenario and a minimum in the "full confidence" scenario, which makes it a reasonable way to quantify our uncertainty in our prediction. There are certainly other methods as well; feel free to experiment with them if desired.

Which five predicted labels are we least confident about? Which five are we most confident about? Print out the associated details (season, player name, stats, etc.) from those seasons.

In [69]:
# answer goes here

gmm = GaussianMixture(n_components=3)
gmm.fit(X_std)

probs = pd.DataFrame(gmm.predict_proba(X_std))

entropies = pd.DataFrame(entr(gmm.predict_proba(X_std)))

In [49]:
entropies

,0,1,2
0,0.059003,1.633222e-02,0.015970
1,0.015725,9.889136e-03,0.035869
2,0.131866,4.275594e-02,0.014039
3,0.000158,8.777238e-06,0.000015
4,0.006472,1.160051e-01,0.034731
...,...,...,...
1758,0.000013,9.428235e-27,0.000149
1759,0.083947,2.418235e-02,0.014938
1760,0.001481,3.519259e-07,0.000171
1761,0.001226,3.543224e-01,0.337325


In [50]:
probs

,0,1,2
0,0.013768,9.835314e-01,0.002700
1,0.002650,9.900613e-01,0.007288
2,0.041413,9.562738e-01,0.002313
3,0.000014,6.136264e-07,0.999985
4,0.000927,3.443692e-02,0.964636
...,...,...,...
1758,0.999987,1.471177e-28,0.000013
1759,0.021993,9.755154e-01,0.002492
1760,0.000171,1.984313e-08,0.999829
1761,0.000138,4.721763e-01,0.527686


In [86]:
class_weights = {0: 1/3, 1: 1/3, 2: 1/3}

entropies['weighted'] = entropies[class_weights.keys()].apply(
    lambda x: np.sum([x[i] * class_weights[i] for i in class_weights]), 
    axis=1)

most_certain_index = entropies.sort_values(by='weighted').index[0]
least_certain_index = entropies.sort_values(by='weighted', ascending=False).index[0]

In [88]:
least_certain_prob = probs.iloc[least_certain_index]
most_certain_prob = probs.iloc[most_certain_index]

print(least_certain_prob)
print(most_certain_prob)

0    0.371660
1    0.327138
2    0.301202
Name: 1647, dtype: float64
0     1.000000e+00
1    1.317536e-221
2    1.322268e-222
Name: 1292, dtype: float64
